One of the main problems with messy data is: how do you know if it's messy or not?

We're going to use the NYC 311 service request dataset again here, since it's big and a bit unwieldy.

In [26]:
# The usual preamble
%matplotlib inline

from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
# Make the graphs a bit prettier, and bigger
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

In [38]:
requests = pd.read_csv(join('..', 'data', 'service-requests.zip'), compression='zip')

In [39]:
requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205699 entries, 0 to 205698
Data columns (total 53 columns):
Unique Key                        205699 non-null int64
Created Date                      205699 non-null object
Closed Date                       177647 non-null object
Agency                            205699 non-null object
Agency Name                       205699 non-null object
Complaint Type                    205699 non-null object
Descriptor                        198646 non-null object
Location Type                     158071 non-null object
Incident Zip                      193820 non-null object
Incident Address                  159835 non-null object
Street Name                       159824 non-null object
Cross Street 1                    114730 non-null object
Cross Street 2                    113886 non-null object
Intersection Street 1             32093 non-null object
Intersection Street 2             31937 non-null object
Address Type                      195

### How do we know if it's messy? 

To get a sense for whether a column has problems, you can use `.nunique()`, `.describe()` or `.value_counts()` to look at all its values, or plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

* Some have been parsed as strings, and some as floats
* There are `nan`s 
* Some of the zip codes are `29616-0759` or `83`
* There are some N/A values that pandas didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

* Normalize 'N/A' and 'NO CLUE' into regular nan values
* Look at what's up with the 83, and decide what to do
* Make everything strings

In [49]:
requests['Incident Zip'].unique()

array([nan, '10037', '10030', '11377', '11105', '10453', '11204', '10459',
       '11232', '10038', '10016', '11234', '11235', '11423', '11368',
       '10040', '10001', '11415', '11229', '11432', '11103', '10012',
       '11375', '11367', '11213', '10003', '10463', '10467', '10465',
       '11355', '11205', '11236', '11218', '11434', '11208', '11212',
       '11385', '10460', '10468', '11226', '11207', '10010', '11417',
       '10458', '10024', '11358', '11222', '11360', '11206', '11209',
       '11378', '10027', '10456', '11215', '11220', '11210', '11102',
       '11416', '11412', '11691', '10022', '11219', '11233', '10451',
       '10469', '10023', '10454', '11201', '10036', '11419', '11249',
       '10075', '10019', '11214', '11237', '11225', '10306', '11372',
       '10031', '11238', '11357', '11354', '11373', '11421', '10462',
       '10314', '10457', '10034', '10065', '10282', '11228', '10025',
       '10303', '10018', '11694', '11216', '11217', '10002', '10466',
       '10305',

### Fixing the missing values and string/float confusion

We can pass a `na_values` option to `pd.read_csv` to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.

In [1]:
na_values = ['UNKNOWN', np.nan, '0']
requests = pd.read_csv('../data/service-requests.zip', compression='zip', 
                       na_values=na_values, dtype={'Incident Zip': str})

/Users/Stefan/.virtualenvs/ga_ds/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
requests['Incident Zip'].drop_duplicates().sort_values()

26547          00000
121000        000000
6780           00083
31795          06902
19192          07006
80458          07011
200859         07024
203102    07030-5773
39457          07047
12693          07054
8609           07073
170733         07086
92967          07087
125195         07093
1332           07114
122154         07302
35126          07305
26677          07310
76332          07607
34112          07666
67483          07758
518            08837
25216          10000
25             10001
219            10002
             ...    
38040          11959
95010          11963
156247         12222
127685         14009
196510         14221
172724         14901
1432      18773-9460
131571         19044
80576          23285
15420          23833
19816          30084
27705          31131
85190          32231
86752     32255-1268
100346         33021
85388          33024
13639          34653
182093    40290-1094
172737         45040
169855         60062
24813          60077
116258    752

### What's up with the dashes?

Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [75]:
requests.loc[requests['Incident Zip'].str.contains('-').fillna(False), 'Incident Zip']

1432      18773-9460
86752     32255-1268
116258    75266-9023
182093    40290-1094
185198    89120-6224
203102    07030-5773
Name: Incident Zip, dtype: object

Let's get rid of the part after the hyphen.

In [76]:
requests['Incident Zip'] = requests['Incident Zip'].str.split('-').str.get(0)

In [77]:
requests.loc[requests['Incident Zip'].str.contains('-').fillna(False), 'Incident Zip']

Series([], Name: Incident Zip, dtype: object)

In [80]:
requests['Incident Zip'].str.len().value_counts()

5.0    193818
6.0         1
Name: Incident Zip, dtype: int64

In [46]:
requests[requests['Incident Zip'] == '00000']

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Park Facility Name,Park Borough,School Name,School Number,School Region,School Code,School Phone Number,School Address,School City,School State,School Zip,School Not Found,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
42600,26529313,10/22/2013 02:51:06 PM,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint,NaN,00000,EWR EWR,EWR,NaN,NaN,NaN,NaN,NaN,NEWARK,NaN,NaN,Assigned,12/07/2013 09:53:51 AM,10/23/2013 09:54:43 AM,0 Unspecified,Unspecified,NaN,NaN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60843,26507389,10/17/2013 05:48:44 PM,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint,Street,00000,1 NEWARK AIRPORT,NEWARK AIRPORT,NaN,NaN,NaN,NaN,NaN,NEWARK,NaN,NaN,Assigned,12/02/2013 11:59:46 AM,10/18/2013 12:01:08 PM,0 Unspecified,Unspecified,NaN,NaN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,N,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This looks bad to me. Let's set these to nan.

In [47]:
zero_zips = requests['Incident Zip'] == '00000'
requests.loc[zero_zips, 'Incident Zip'] = np.nan

In [78]:
unique_zips = requests['Incident Zip'].unique()
unique_zips.sort()
unique_zips

array([nan, '00000', '000000', '00083', '06902', '07006', '07011', '07024',
       '07030', '07047', '07054', '07073', '07086', '07087', '07093',
       '07114', '07302', '07305', '07310', '07607', '07666', '07758',
       '08837', '10000', '10001', '10002', '10003', '10004', '10005',
       '10006', '10007', '10009', '10010', '10011', '10012', '10013',
       '10014', '10016', '10017', '10018', '10019', '10020', '10021',
       '10022', '10023', '10024', '10025', '10026', '10027', '10028',
       '10029', '10030', '10031', '10032', '10033', '10034', '10035',
       '10036', '10037', '10038', '10039', '10040', '10041', '10044',
       '10065', '10069', '10075', '10096', '10103', '10107', '10112',
       '10118', '10119', '10128', '10129', '10153', '10271', '10278',
       '10280', '10281', '10282', '10301', '10302', '10303', '10304',
       '10305', '10306', '10307', '10308', '10309', '10310', '10312',
       '10314', '10451', '10452', '10453', '10454', '10455', '10456',
       '10457'

Amazing! This is much cleaner. There's something a bit weird here, though -- I looked up 77056 on Google maps, and that's in Texas.

Let's take a closer look:

In [5]:
zips = requests['Incident Zip']
# Let's say the zips starting with '0' and '1' are okay, for now. (this isn't actually true -- 13221 is in Syracuse, and why?)
is_close = zips.str.startswith('0') | zips.str.startswith('1')
# There are a bunch of NaNs, but we're not interested in them right now, so we'll say they're False
is_far = ~(is_close) & zips.notnull()

In [50]:
zips[is_far]

12102    77056
13450    70711
29136    77092
30939    55164
44008    90010
47048    23541
57636    92123
71001    92123
71834    23502
80573    61702
85821    29616
89304    35209
94201    41042
Name: Incident Zip, dtype: object

In [51]:
requests[is_far][['Incident Zip', 'Descriptor', 'City']].sort('Incident Zip')

,Incident Zip,Descriptor,City
71834,23502,Harassment,NORFOLK
47048,23541,Harassment,NORFOLK
85821,29616,Debt Not Owed,GREENVILLE
89304,35209,Harassment,BIRMINGHAM
94201,41042,Harassment,FLORENCE
30939,55164,Harassment,ST. PAUL
80573,61702,Billing Dispute,BLOOMIGTON
13450,70711,Contract Dispute,CLIFTON
12102,77056,Debt Not Owed,HOUSTON
29136,77092,False Advertising,HOUSTON


Okay, there really are requests coming from LA and Houston! Good to know. Filtering by zip code is probably a bad way to handle this -- we should really be looking at the city instead.

In [52]:
requests['City'].str.upper().value_counts()

BROOKLYN            31662
NEW YORK            22664
BRONX               18438
STATEN ISLAND        4766
JAMAICA              2246
FLUSHING             1803
ASTORIA              1568
RIDGEWOOD            1073
CORONA                707
OZONE PARK            693
LONG ISLAND CITY      678
FAR ROCKAWAY          652
ELMHURST              647
WOODSIDE              609
EAST ELMHURST         562
...
MELVILLE                  1
PORT JEFFERSON STATION    1
NORWELL                   1
EAST ROCKAWAY             1
BIRMINGHAM                1
ROSLYN                    1
LOS ANGELES               1
MINEOLA                   1
JERSEY CITY               1
ST. PAUL                  1
CLIFTON                   1
COL.ANVURES               1
EDGEWATER                 1
ROSELYN                   1
CENTRAL ISLIP             1
Length: 100, dtype: int64

It looks like these are legitimate complaints, so we'll just leave them alone.

# 7.5 Putting it together

Here's what we ended up doing to clean up our zip codes, all together:

In [53]:
na_values = ['NO CLUE', 'N/A', '0']
requests = pd.read_csv('../data/311-service-requests.csv', 
                       na_values=na_values, 
                       dtype={'Incident Zip': str})

In [54]:
def fix_zip_codes(zips):
    # Truncate everything to length 5 
    zips = zips.str.slice(0, 5)
    
    # Set 00000 zip codes to nan
    zero_zips = zips == '00000'
    zips[zero_zips] = np.nan
    
    return zips

In [55]:
requests['Incident Zip'] = fix_zip_codes(requests['Incident Zip'])

In [56]:
requests['Incident Zip'].unique()

array(['11432', '11378', '10032', '10023', '10027', '11372', '11419',
       '11417', '10011', '11225', '11218', '10003', '10029', '10466',
       '11219', '10025', '10310', '11236', nan, '10033', '11216', '10016',
       '10305', '10312', '10026', '10309', '10036', '11433', '11235',
       '11213', '11379', '11101', '10014', '11231', '11234', '10457',
       '10459', '10465', '11207', '10002', '10034', '11233', '10453',
       '10456', '10469', '11374', '11221', '11421', '11215', '10007',
       '10019', '11205', '11418', '11369', '11249', '10005', '10009',
       '11211', '11412', '10458', '11229', '10065', '10030', '11222',
       '10024', '10013', '11420', '11365', '10012', '11214', '11212',
       '10022', '11232', '11040', '11226', '10281', '11102', '11208',
       '10001', '10472', '11414', '11223', '10040', '11220', '11373',
       '11203', '11691', '11356', '10017', '10452', '10280', '11217',
       '10031', '11201', '11358', '10128', '11423', '10039', '10010',
       '11209',

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  